# Widget Experiments for Creating UI to control Label Placement

In [ ]:
import json
import pathlib

import ipywidgets as W
import traitlets as T

import ipyelk


class NodeLabelPlacement(W.HBox):
    horizontal = T.Enum(values=["left", "center", "right"], allow_none=True)
    h_priority = T.Bool(allow_none=True)
    vertical = T.Enum(values=["top", "center", "bottom"], allow_none=True)
    inside = T.Enum(values=["inside", "outside"], default_value="inside")
    value = T.Unicode(allow_none=True)

    def __init__(self, *args, **kwargs):
        """Simple widget to help format the ELK Node Label Placement. 
        
        Hints for where node labels are to be placed; if empty, the node 
        label’s position is not modified.
        """
        super().__init__(*args, **kwargs)

        self.horizontal_options = W.RadioButtons(
            description="Horizontal",
            options=(("Left", "left"), ("Center", "center"), ("Right", "right"),),
        )
        self.vertical_options = W.RadioButtons(
            description="Vertical",
            options=(("Top", "top"), ("Center", "center"), ("Bottom", "bottom"),),
        )
        self.inside_options = W.Checkbox(description="Inside")
        self.horizontal_priority_options = W.Checkbox(description="Horizontal Priority")

        T.link((self, "horizontal"), (self.horizontal_options, "value"))
        T.link((self, "vertical"), (self.vertical_options, "value"))
        T.link((self, "h_priority"), (self.horizontal_priority_options, "value"))

        def _handle_inside_change(change):
            self.inside = "inside" if self.inside_options.value is True else "outside"

        self.inside_options.observe(_handle_inside_change, "value")

        self.children = [
            self.horizontal_options,
            self.vertical_options,
            W.VBox([self.inside_options, self.horizontal_priority_options,]),
        ]

        self._update_value()

    @T.observe("horizontal", "vertical", "inside", "h_priority")
    def _update_value(self, change=None):
        self.inside_options.value = self.inside == "inside"
        self.horizontal_priority_options.value = self.h_priority
        options = []
        if self.horizontal:
            options.append(f"H_{self.horizontal.upper()}")
        if self.vertical:
            options.append(f"V_{self.vertical.upper()}")
        if self.inside:
            options.append(self.inside.upper())
        if self.h_priority:
            options.append("H_PRIORITY")
        if options:
            self.value = f"[{' '.join(options)}]"
        else:
            self.value = None


nlp = NodeLabelPlacement()
nlp

### ELKJSON build acceptable value for the label placement

In [ ]:
nlp.value

In [ ]:
nlp.horizontal = "center"
nlp.vertical = "top"

In [ ]:
simple = ipyelk.ElkDiagram()


def update_layout_options(node, options):
    "Recurse through the nodes and update layout options"
    for label in node.get("labels", []):
        if "layoutOptions" in label:
            label["layoutOptions"].update(options)
        else:
            label["layoutOptions"] = options

        label.pop("x", None)
        label.pop("y", None)
        label["height"] = 15
        label["width"] = 5.4 * len(label["text"])
    for child in node.get("children", []):
        update_layout_options(child, options)


def update_json(change):
    elk_json = json.loads(pathlib.Path("simple.json").read_text(encoding="utf-8"))

    layoutOptions = {"nodeLabels.placement": nlp.value}

    #     elk_json["layoutOptions"] = layoutOptions
    #     elk_json["children"][0]["labels"] = [
    #         {'text': 'kernel', 'id': 'kernel_label', "layoutOptions": layoutOptions, "width":30, "height":15},
    #         {'text': "line 2", 'id': 'kernel_label3', "layoutOptions": layoutOptions, "width":30, "height":15}
    #     ]
    update_layout_options(elk_json, layoutOptions)
    simple.value = elk_json


nlp.observe(update_json, "value")
update_json(None)
W.HBox([simple, nlp], layout={"height": "400px"})

In [ ]:
from IPython.display import JSON

JSON(simple.value)